In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf,SparkContext

### Boilerplate - Spark Session & HDFS Access

In [2]:
cwd = os.getcwd()
for part in cwd.split('/'):
    if part.lower().startswith('edureka'):
        user_id = part.title()
app_name = '{0} : Random Forest'.format(user_id)
app_name

'Edureka_121039 : Random Forest'

In [3]:
spark = SparkSession.builder.appName(app_name).getOrCreate()
sparkContext = spark.sparkContext
sqlContext = SQLContext(sparkContext)

In [4]:
def get_hdfs_filepath(file_name):
    my_hdfs = '/user/{0}'.format(user_id.lower())
    return os.path.join(my_hdfs, file_name)

### Dataset

In [5]:
TITANIC_CSV = get_hdfs_filepath('train_titanic.csv')

### ML Imports

In [11]:
# Import packages
from numpy import array
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder, VectorAssembler, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import *

In [6]:
titanic = spark.read.csv(TITANIC_CSV,inferSchema=True,header=True)
titanic.show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [7]:
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [10]:
# String to float on some columns of the dataset : creates a new dataset
titanic = titanic.select(col("Survived"),
                         col("Sex"),
                         col("Embarked"),
                         col("Pclass").cast("float"),
                         col("Age").cast("float"),
                         col("SibSp").cast("float"),
                         col("Fare").cast("float"))

In [12]:
# dropping null values
titanic = titanic.dropna()

In [13]:
# Spliting in train and test set. Beware : It sorts the dataset
(traindf, testdf) = titanic.randomSplit([0.7,0.3])

In [14]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
genderIndexer = StringIndexer(inputCol="Sex", outputCol="indexedSex")
embarkIndexer = StringIndexer(inputCol="Embarked", outputCol="indexedEmbarked")
surviveIndexer = StringIndexer(inputCol="Survived", outputCol="indexedSurvived")

In [15]:
# One Hot Encoder on indexed features
genderEncoder = OneHotEncoder(inputCol="indexedSex", outputCol="sexVec")
embarkEncoder = OneHotEncoder(inputCol="indexedEmbarked", outputCol="embarkedVec")

In [16]:
# Create the vector structured data (label,features(vector))
assembler = VectorAssembler(inputCols=["Pclass","sexVec","Age","SibSp","Fare","embarkedVec"],outputCol="features")
 

In [17]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedSurvived", featuresCol="features")

In [18]:
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[surviveIndexer, genderIndexer, embarkIndexer, genderEncoder,embarkEncoder, assembler, rf]) # genderIndexer,embarkIndexer,genderEncoder,embarkEncoder,

In [19]:
# Train model.  This also runs the indexers.
model = pipeline.fit(traindf)
 
# Predictions
predictions = model.transform(testdf)
 
# Select example rows to display.
predictions.columns 

['Survived',
 'Sex',
 'Embarked',
 'Pclass',
 'Age',
 'SibSp',
 'Fare',
 'indexedSurvived',
 'indexedSex',
 'indexedEmbarked',
 'sexVec',
 'embarkedVec',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [20]:
# Select example rows to display.
predictions.select("prediction", "Survived", "features").show(5)

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       1.0|       0|[3.0,0.0,9.0,1.0,...|
|       1.0|       0|[2.0,0.0,24.0,0.0...|
|       1.0|       0|[3.0,0.0,2.0,0.0,...|
|       0.0|       0|[3.0,0.0,2.0,4.0,...|
|       0.0|       0|[3.0,0.0,8.0,3.0,...|
+----------+--------+--------------------+
only showing top 5 rows



In [21]:
# Select (prediction, true label) and compute test error
predictions = predictions.select(col("Survived").cast("Float"),col("prediction"))
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.178082


In [22]:
rfModel = model.stages[6]
print(rfModel)  # summary only

RandomForestClassificationModel (uid=rfc_4f58b5f9be74) with 20 trees


In [23]:
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)

Accuracy = 0.821918


In [24]:
evaluatorf1 = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="f1")
f1 = evaluatorf1.evaluate(predictions)
print("f1 = %g" % f1)

f1 = 0.816024


In [25]:
evaluatorwp = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="weightedPrecision")
wp = evaluatorwp.evaluate(predictions)
print("weightedPrecision = %g" % wp)

weightedPrecision = 0.824275


In [26]:
evaluatorwr = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="weightedRecall")
wr = evaluatorwr.evaluate(predictions)
print("weightedRecall = %g" % wr)

weightedRecall = 0.821918


In [28]:
# close sparkcontext
sparkContext.stop()